# **Установка нужных версий для работы**

In [ ]:
!pip install polars==0.20.19 scipy==1.13.0 scikit-learn==1.4.1.post1 numpy==1.26.4 pandas==2.0.3 rapidfuzz~=3.8.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 78.6 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.1
    Uninstalling pandas-2.2.1:
      Successfully uninstalled pandas-2.2.1


# **Импорты**

In [ ]:
import polars as pl
import numpy as np
import pandas as pd
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from rapidfuzz import process
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

# **Директория с датасетом**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DATASET_DIR = 'drive/MyDrive/dataset'

# **Простой алгоритм**

In [ ]:
def weighted_rating(count, avg, quantile: pl.DataFrame, mean: pl.DataFrame):
    quantile = float(quantile.head(1).row(0)[0])
    return (count / (count + quantile) * avg) + (quantile / (quantile + count) * mean)

In [ ]:
def get_content_simple_recs(movies_metadata: pl.DataFrame, head_count: int) -> pl.DataFrame:
    """
    Функция, возвращающие общие рекомендации по фильмам, без уточняющих характеристик.
    :param movies_metadata:
    :param head_count: Количество фильмов, которые надо возвратить в итоговом дата фрейме
    :return:
    """
    # Получаем среднее значение оценок под фильмами
    mean: pl.DataFrame = movies_metadata.select('vote_average').mean()
    # Агрегируем столбцы этого DataFrame до квантиля 0.90.
    quantile: pl.DataFrame = movies_metadata.select('vote_count').quantile(0.90)
    # Получаем дата фрейм из значений количества голосов больше квантиля
    q_movies: pl.DataFrame = movies_metadata.filter(pl.col('vote_count') >= quantile)
    # Дописываем столбец со взвешенным рейтингом, название столбца - 'score'
    q_movies: pl.DataFrame = q_movies.with_columns(
        weighted_rating(
            pl.col('vote_count'),
            pl.col('vote_average'),
            quantile,
            mean
        ).alias('Оценка алгоритма'),
        pl.col('title').alias('Название'),
        pl.col('vote_count').alias('Количество оценок'),
        pl.col('vote_average').alias('Средняя оценка')
    )
    # Сортируем дата фрейм по убыванию взвешенного рейтинга
    q_movies: pl.DataFrame = q_movies.sort('Оценка алгоритма', descending=True)
    return q_movies[['Название', 'Количество оценок', 'Средняя оценка', 'Оценка алгоритма']].head(head_count)

# **По содержанию**

# TF-IDF

In [ ]:
def get_content_tfidf_recs(movies_metadata: pl.DataFrame, title: str) -> pl.DataFrame:
    tfidf: TfidfVectorizer = TfidfVectorizer(stop_words='english')
    movies_metadata: pl.DataFrame = movies_metadata.with_columns(
        pl.col('overview').fill_null('')
    ).drop('vote_average', 'vote_count')
    overview_series: pl.Series = movies_metadata.select('overview').to_series()

    # Составляем матрицу TF-IDF
    from scipy.sparse import csr_matrix
    tfidf_matrix: csr_matrix = tfidf.fit_transform(overview_series)
    cosine_sim: np.ndarray = linear_kernel(tfidf_matrix, tfidf_matrix)
    movies_metadata = movies_metadata.with_row_index("index")
    # Получаем индекс фильма, название которого совпадает с заданным
    expr: pl.Expr = pl.all_horizontal(
        pl.col('title') == title
    )
    idx = movies_metadata.row(by_predicate=expr, named=True)['index']
    # Получаем попарную схожесть всех фильмов с фильмом, который нам дан
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Сортируем фильмы на основании очков схожести
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Получаем очки для 10 самых похожих фильмов
    sim_scores = sim_scores[1:11]

    # Получаем индексы фильмов
    movie_indices = [i[0] for i in sim_scores]
    return movies_metadata.select('title')[movie_indices]

# По ключевым словам

In [ ]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan


def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        if len(names) > 3:
            names = names[:3]
        return names

    return []


def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        # Проверяем что директор существует. Если нет, возвращаем пустую строку
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''


def get_recommendations(title, metadata: pd.DataFrame, indices, cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return metadata['title'].iloc[movie_indices]


def create_soup(x):
    return ' '.join(str(x['keywords'])) + ' ' + ' '.join(str(x['cast'])) + ' ' + str(x['director']) + ' ' + ' '.join(str(x['genres']))


def get_content_keywords_recs(
        title: str
) -> pd.DataFrame:
    metadata = pd.read_csv(f'{DATASET_DIR}/movies_metadata.csv', low_memory=False)
    credits = pd.read_csv(f'{DATASET_DIR}/credits.csv', nrows=15000)
    keywords = pd.read_csv(f'{DATASET_DIR}/keywords.csv')
    # Сломанные записи
    metadata = metadata.drop([19730, 29503, 35587])
    keywords['id'] = keywords['id'].astype('int')
    credits['id'] = credits['id'].astype('int')
    metadata['id'] = metadata['id'].astype('int')
    metadata = metadata.merge(credits, on='id')
    metadata = metadata.merge(keywords, on='id')

    features = ['cast', 'crew', 'keywords', 'genres']

    for feature in features:
        metadata[feature] = metadata[feature].apply(literal_eval)
    metadata['director'] = metadata['crew'].apply(get_director)
    for feature in features:
        metadata[feature] = metadata[feature].apply(get_list)
    for feature in features:
        metadata[feature] = metadata[feature].apply(clean_data)
    metadata['soup'] = metadata.apply(create_soup, axis=1)
    count = CountVectorizer(stop_words='english')
    count_matrix = count.fit_transform(metadata['soup'])
    cosine_sim = cosine_similarity(count_matrix, count_matrix)
    metadata = metadata.reset_index()
    indices = pd.Series(metadata.index, index=metadata['title'])
    return get_recommendations(title, metadata, indices, cosine_sim)

# **Коллаборативная фильтрация**

# K ближайших соседей

In [ ]:
def get_collaborative_knn_recs(movie_names: pd.DataFrame, ratings_data: pd.DataFrame, movie_name: str):
    # Дата фрейм с названием фильма и его жанром
    movie_names = movie_names[['title', 'genres']]
    # Дата фрейм, в котором
    movies_users: pd.DataFrame = ratings_data.pivot(index=['userId'], columns=['movieId'], values='rating').fillna(0)
    # Преобразовываем в разреженную матрицу (CSR)
    mat_movies_users: csr_matrix = csr_matrix(movies_users.values)
    model_knn: NearestNeighbors = NearestNeighbors(metric='cosine', algorithm='auto', n_neighbors=30, n_jobs=-1)
    model_knn.fit(mat_movies_users)
    movie_index: int = process.extractOne(movie_name, movie_names['title'])[2]
    distances, indices = model_knn.kneighbors(mat_movies_users[movie_index], n_neighbors=20)
    recc_movie_indices: list = sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())),
                                      key=lambda x: x[1])[:0:-1]
    # Список с рекомендациями
    recommend_list = []
    # На каждый индекс рекомендаций
    for val in recc_movie_indices:
        # Добавляем в датафрейм рекомендаций названий фильма и расстояние
        recommend_list.append({'Title': movie_names['title'][val[0]], 'Distance': val[1]})
    # Датафрейм с рекомендациями
    df = pd.DataFrame(recommend_list, index=range(1, 20))
    return df

# SVD

In [ ]:
def get_collaborative_svd_recs(user_id: int, num_recommendations: int):
    movies_df: pd.DataFrame = pd.read_csv(f'{DATASET_DIR}/movies_metadata.csv', low_memory=True, encoding='latin-1')
    movies_df.drop(columns=['budget'])
    movies_df = movies_df.drop([19730, 29503, 35587]).head(80000)
    movies_df.rename(columns={'id': 'movieId'}, inplace=True)
    ratings_df: pd.DataFrame = pd.read_csv(f'{DATASET_DIR}/ratings.csv', encoding='latin-1').head(270000)
    ratings_df['movieId'] = ratings_df['movieId'].astype(int)
    movies_df['movieId'] = movies_df['movieId'].astype(int)
    # Merge the two datasets
    #movies_df.index = movies_df['movieId']
    #ratings_df.index = ratings_df['movieId']
    df: pd.DataFrame = pd.merge(ratings_df, movies_df, on="movieId")
    pivot_table: pd.DataFrame = df.pivot_table(index="userId", columns="title", values="rating")
    pivot_table = pivot_table.dropna(axis='columns', thresh=2)
    overall_mean = pivot_table.mean(axis=1)
    # Fill missing values with the mean rating
    pivot_table.fillna(overall_mean, axis='index', inplace=True)
    from scipy.sparse.linalg import svds
    pivot_np = pivot_table.to_numpy(na_value=2.5)
    U, sigma, Vt = svds(pivot_np, k=1)
    user_item_matrix = sigma * Vt.T
    user_rating_vector = user_item_matrix[user_id - 1]
    similarity_scores = np.corrcoef(user_rating_vector, user_item_matrix)[0, 1:]
    top_movies = np.argsort(similarity_scores)[::-1][:num_recommendations]
    return movies_df.iloc[top_movies]["title"]

# **Песочница**

In [ ]:
movies_df: pl.DataFrame = (pl.read_csv(f'{DATASET_DIR}/movies_metadata.csv', infer_schema_length=100000)
                               .select(pl.col("title", "overview", "vote_average", "vote_count", "genres"))).head(30000)

In [ ]:
user_ratings_df: pl.DataFrame = (pl.read_csv(f'{DATASET_DIR}/ratings_small.csv')
                                     .select(pl.col("userId", "movieId", "rating"))).head(100000)

In [ ]:
print("---Результат работы простейшего алгоритма---")
print(get_content_simple_recs(movies_df, 10))

---Результат работы простейшего алгоритма---
shape: (10, 4)
┌─────────────────────────────┬───────────────────┬────────────────┬──────────────────┐
│ Название                    ┆ Количество оценок ┆ Средняя оценка ┆ Оценка алгоритма │
│ ---                         ┆ ---               ┆ ---            ┆ ---              │
│ str                         ┆ i64               ┆ f64            ┆ f64              │
╞═════════════════════════════╪═══════════════════╪════════════════╪══════════════════╡
│ The Shawshank Redemption    ┆ 8358              ┆ 8.5            ┆ 8.418993         │
│ The Godfather               ┆ 6024              ┆ 8.5            ┆ 8.388892         │
│ The Dark Knight             ┆ 12269             ┆ 8.3            ┆ 8.24839          │
│ Fight Club                  ┆ 9678              ┆ 8.3            ┆ 8.23493          │
│ Pulp Fiction                ┆ 8670              ┆ 8.3            ┆ 8.227582         │
│ Dilwale Dulhania Le Jayenge ┆ 661               ┆ 9.1     

In [ ]:
print("---Результат работы алгоритма на базе TF-IDF---")
print(get_content_tfidf_recs(movies_df, 'The Dark Knight Rises').head(10))

---Результат работы алгоритма на базе TF-IDF---
shape: (10, 1)
┌───────────────────────────────────┐
│ title                             │
│ ---                               │
│ str                               │
╞═══════════════════════════════════╡
│ The Dark Knight                   │
│ Batman Forever                    │
│ Batman Returns                    │
│ Batman: Under the Red Hood        │
│ Batman                            │
│ Batman Unmasked: The Psychology … │
│ Batman Beyond: Return of the Jok… │
│ Batman: Year One                  │
│ Batman: The Dark Knight Returns,… │
│ Batman: Mask of the Phantasm      │
└───────────────────────────────────┘


In [ ]:
print("---Результат работы алгоритма на базе заготовленных ключевых слов---")
print(get_content_keywords_recs('Jumanji').head(10))

---Результат работы алгоритма на базе заготовленных ключевых слов---
552                             The Pagemaster
1953                  Honey, I Shrunk the Kids
1993                             The Rocketeer
2396                               October Sky
4520                         Jurassic Park III
7186                                   Hidalgo
14875                              The Wolfman
8302                 The Invisible Man Returns
662                                  Space Jam
868      Halloween: The Curse of Michael Myers
Name: title, dtype: object


In [ ]:
print("---Результат работы алгоритма KNN ---")
print(get_collaborative_knn_recs(movies_df.to_pandas(), user_ratings_df.to_pandas(), 'Jumanji'))

                                               Title  Distance
1                                      Mr. Wonderful  0.523668
2                         The Silences of the Palace  0.523338
3                                         Tarantella  0.522694
4                                 The Usual Suspects  0.519348
5                                       Mi Vida Loca  0.516944
6                                     Antonia's Line  0.505929
7                             The Man without a Face  0.473188
8                                          Boomerang  0.470424
9                               Sleepless in Seattle  0.466811
10                                         Pinocchio  0.462066
11                       The Amazing Panda Adventure  0.461603
12                                              Nell  0.452125
13                                   Billy's Holiday  0.448570
14                                         Desperado  0.441926
15                                   Colonel Chabert  0

In [ ]:
print("---Результат работы алгоритма SVD ---")
print(get_collaborative_svd_recs(1, 20))

---Результат работы алгоритма SVD ---


<ipython-input-13-07e071467c9f>:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df: pd.DataFrame = pd.read_csv(f'{DATASET_DIR}/movies_metadata.csv', low_memory=True, encoding='latin-1')


2598                                          Lake Placid
852                                               Bye-Bye
870                               For Whom the Bell Tolls
869                                            Wild Reeds
868                                                Venice
867                                              Liebelei
866                                          Mother Night
865                                         Twelfth Night
864                 Halloween: The Curse of Michael Myers
863                                           Baton Rouge
862                                                 1-900
861     The Land Before Time III: The Time of the Grea...
860                                        Talk of Angels
859                                           Bulletproof
858                                                 Bogus
857                                         Sweet Nothing
856                                    The Trigger Effect
855           

/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
